In [2]:
# using a python genomics package called hail. first, importing it 

import hail as hl
#hl.init()



Loading BokehJS ...

In [3]:
# you need to hand-make a reference genome so this is that 

ahqt = hl.ReferenceGenome(name="ahqt",contigs=["Chr_01","Chr_02"],lengths={"Chr_01": 249250621, "Chr_02": 155270560, "Y": 59373566})

ahqt.from_fasta_file(name = "AHQTv1",
                     fasta_file = "/home/colette_berg/resources/reference_genomes/AHQT/Mimulus_guttatus_var_AHQT1.mainGenome.fa",
                     index_file = "/home/colette_berg/resources/reference_genomes/AHQT/Mimulus_guttatus_var_AHQT1.mainGenome.fa.fai")

Initializing Hail with default parameters...
SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/home/colette_berg/.conda/envs/YNP/lib/python3.9/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://df1f36967732:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.126-ee77707f4fab
LOGGING: writing to /scratch/colette/demux_novogene_lane_2023/YNP_GWAS/scripts/hail

ReferenceGenome(name=AHQTv1, contigs=['Chr_01', 'Chr_02', 'Chr_03', 'Chr_04', 'Chr_05', 'Chr_06', 'Chr_07', 'Chr_08', 'Chr_09', 'Chr_10', 'Chr_11', 'Chr_12', 'Chr_13', 'Chr_14'], lengths={'Chr_01': 14785156, 'Chr_02': 23244023, 'Chr_03': 24937297, 'Chr_04': 22086957, 'Chr_05': 23317283, 'Chr_06': 22883558, 'Chr_07': 20421051, 'Chr_08': 26741851, 'Chr_09': 24050817, 'Chr_10': 24958264, 'Chr_11': 20167067, 'Chr_12': 25528010, 'Chr_13': 24206043, 'Chr_14': 29734813}, x_contigs=[], y_contigs=[], mt_contigs=[], par=[])

In [4]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()



Loading BokehJS ...

In [4]:
# importing the vcf with the reference genome I just created, then writing a data table 
dataDir="/scratch/colette/demux_novogene_lane_2023/YNP_GWAS/data/"

hl.import_vcf(dataDir + 'YNP_thinned_biallelic.recode.vcf',
                        reference_genome='AHQTv1').write('YNP.mt', overwrite=True)



2024-01-10 19:22:51.131 Hail: INFO: scanning VCF for sortedness...
2024-01-10 19:22:57.139 Hail: INFO: Coerced sorted VCF - no additional import work to do
2024-01-10 19:23:01.943 Hail: INFO: wrote matrix table with 24542 rows and 86 columns in 1 partition to YNP.mt


In [5]:
# now reading in that data table 

mt = hl.read_matrix_table('YNP.mt')


In [6]:
rel = hl.pc_relate(mt.GT,
             min_individual_maf = .1,
             k=10,
             statistics = 'kin')

2024-01-10 19:23:19.401 Hail: INFO: hwe_normalize: found 18393 variants after filtering out monomorphic sites.
2024-01-10 19:23:21.294 Hail: INFO: pca: running PCA with 10 components...
2024-01-10 19:23:24.882 Hail: INFO: wrote table with 0 rows in 0 partitions to /tmp/persist_TablejUh4m9V3zg
2024-01-10 19:23:29.434 Hail: INFO: wrote matrix with 24542 rows and 86 columns as 6 blocks of size 4096 to /tmp/BMbzSja6itfEI5m9WN0qKT
2024-01-10 19:23:31.027 Hail: INFO: wrote matrix with 11 rows and 24542 columns as 6 blocks of size 4096 to /tmp/pcrelate-write-read-UKSVpAC5HuzO7nqm21GOBE.bm
2024-01-10 19:23:31.245 Hail: INFO: wrote matrix with 24542 rows and 86 columns as 6 blocks of size 4096 to /tmp/pcrelate-write-read-Ivw8gNfXslcfy1z3DDwE9s.bm
2024-01-10 19:23:31.717 Hail: INFO: wrote matrix with 86 rows and 86 columns as 1 block of size 4096 to /tmp/pcrelate-write-read-2rGktlD1UIKnfKBIJsNvJE.bm
2024-01-10 19:23:32.244 Hail: INFO: wrote matrix with 86 rows and 86 columns as 1 block of size 4

In [7]:
IBD = hl.identity_by_descent(mt)

2024-01-10 19:23:47.525 Hail: INFO: Ordering unsorted dataset with network shuffle
2024-01-10 19:23:48.466 Hail: INFO: wrote table with 3655 rows in 1 partition to /tmp/persist_TablemD1ofOzHZd


In [8]:
#import pandas as pd
#import numpy as np

#df = pd.DataFrame(rel)


df_ibd = IBD.to_pandas()

In [9]:
df_ibd.to_csv("IBD.csv")